In [1]:
import os
import pandas as pd

from sklearn.metrics import f1_score

# Question 8

You will not get anywhere with your models, if you cannot make them read clinical data…
To answer the next question, you will have to write code processing a set of HL7 messages (saved as plain text files).  The messages are located in the Clinical_HL7_Samples/ subfolders (loaded from a public HL7 test repository)


In what year was the youngest male patient born?

In [2]:
# Define a function to design the desired structure to store the HL7 messages:
# {folder_name:{message_name:message}}

def make_hl7_dict(root_folder):
    
    hl7_dict = {}
    
    # Iterate through folders in root folder
    for folder_name in os.listdir(root_folder):
        folder_path = os.path.join(root_folder, folder_name)
        
        # Checking if it's a directory
        if os.path.isdir(folder_path):
            folder_dict = {}
            for txt_file_name in os.listdir(folder_path):
                txt_file_path = os.path.join(folder_path, txt_file_name)
                
                # Retrieving text files exclusively
                if txt_file_name.endswith('.txt'):
                    with open(txt_file_path, 'r') as f:
                        message = f.read()
                        folder_dict[txt_file_name] = message
            hl7_dict[folder_name] = folder_dict
            
    return hl7_dict


# Using absolute path of root folder
root_folder = "/Users/jamesliounis/Documents/Harvard SM DS/NCE512/NCE_512_Problem_Sets/NCE512_PS1/Clinical_HL7_Samples"

# Invoking previously defined function on root folder 
hl7_dict = make_hl7_dict(root_folder)

# Create list to store unique messages
unique_messages = []

# Iterate through the dictionaries that store the messages
for message_dict in hl7_dict.values():
    # Iterate through  messages
    for message in message_dict.values():
        # Retrieve individual messages, splitting on new line as per Hl7 guidelines
        unique_messages.append(message.split("n"))
        

# Define a function to extract DOB from HL7 string
def extract_dob(hl7_string):
    hl7_fields = hl7_string.split("|")
    dob_field = None
    for field in hl7_fields:
        if len(field) == 8 and field.isdigit():
            dob_field = field
            break
    return dob_field


# Create list to store full DOB, from which we will retrieve YOB
DOBs = []

# Iterate through unique messages
for message in unique_messages:
    
    # Iterate through individual HL7 strings
    for hl7_string in message:
        
        # Invoke previously defined function to retrieve DOB
        DOBs.append(extract_dob(hl7_string))
        
# Extracting first 4 digits (YOB)
YOBs = [int(dob[:4]) for dob in DOBs if dob is not None]

# Retrieving largest value
YOB_of_youngest_male = max(YOBs)

print("The youngest male patient was born in", YOB_of_youngest_male,".")


The youngest male patient was born in 2013 .


# Question 9

Mining logical rule models from the raw data can help us discover extremely efficient, interpretable patterns, often leading to optimal decision-making. To work on this problem, use COVID_Data_Small.xlsx file, provided with this homework – it contains “close to real” data on a large number of ED patients during the first COVID outbreak in Spring of 2020. The target variable is Outcome_48Hours_Dispo – whether the patient was placed on high oxygen support within 48 hours since hospital admission (disposition). Our goal is to use the other variables to discover, which ones of them, and under what conditions, are closely correlated with his outcome. 

When running logical rule models with the code explained in the class, what COVID risk pattern is identified by the most accurate rule?


In [3]:
PATH = "/Users/jamesliounis/Documents/Harvard SM DS/NCE512/NCE_512_Problem_Sets/NCE512_PS1/COVID_Data_Small.xlsx"

covid_df = pd.read_excel(PATH)

In [4]:
# EDA

covid_df.head()

,Arrival_Date,AgeYears,Temperature,BMI,AvgReading_Neuts_pct,Respiration_Rate,HasConnectiveTissueDisorderFLG,O2_Saturation,MaleFLG,Outcome_48Hours_Dispo,Rand
0,2020-05-28,75.833333,98.0,36.3,75.6,98,1,99,1,0,0.055920
1,2020-03-10,0.916667,102.0,34.6,71.2,70,0,98,0,1,0.112461
2,2020-05-29,61.083333,97.5,25.3,68.4,61,1,99,1,0,0.031409
3,2020-05-11,56.166667,97.9,30.3,96.5,56,0,98,1,1,0.720525
4,2020-06-19,41.916667,97.0,34.1,53.7,56,0,100,0,0,0.047407


In [5]:
covid_df.describe()

,AgeYears,Temperature,BMI,AvgReading_Neuts_pct,Respiration_Rate,HasConnectiveTissueDisorderFLG,O2_Saturation,MaleFLG,Outcome_48Hours_Dispo,Rand
count,6004.000000,6004.000000,6004.00000,6004.000000,6004.000000,6004.000000,6004.000000,6004.000000,6004.000000,6004.000000
mean,57.190734,98.263175,29.60708,70.416039,19.455696,0.323784,97.106929,0.500500,0.392572,0.223282
std,20.197951,1.065284,50.80480,13.456411,4.838267,0.467958,2.625410,0.500041,0.488363,0.286020
min,0.000000,85.000000,7.60000,4.000000,0.000000,0.000000,18.000000,0.000000,0.000000,0.000002
25%,41.833333,97.700000,23.60000,61.800000,17.000000,0.000000,96.000000,0.000000,0.000000,0.034448
50%,59.333333,98.200000,27.50000,71.900000,18.000000,0.000000,97.000000,1.000000,0.000000,0.069807
75%,72.666667,98.700000,32.60000,80.800000,20.000000,1.000000,99.000000,1.000000,1.000000,0.376334
max,106.500000,105.100000,3912.70000,97.000000,98.000000,1.000000,100.000000,1.000000,1.000000,0.999802


In [6]:
covid_df.dtypes

Arrival_Date                       object
AgeYears                          float64
Temperature                       float64
BMI                               float64
AvgReading_Neuts_pct              float64
Respiration_Rate                    int64
HasConnectiveTissueDisorderFLG      int64
O2_Saturation                       int64
MaleFLG                             int64
Outcome_48Hours_Dispo               int64
Rand                              float64
dtype: object

In [7]:
covid_df.dtypes
covid_df = covid_df.applymap(lambda x: pd.to_numeric(x, errors='ignore'))

We can see here that all data types are the appropriate ones. The only issue is that arrival date cannot be used in a logical rule model. Hence, we have to drop it. 

In [8]:
# Total record count:

nRecords = len(covid_df)
print("The dataframe contains",nRecords,'records.')

The dataframe contains 6004 records.


In [9]:
# Specify original features to choose from
features = [i for i in covid_df.columns]

# Remove the dependent variable from these features
features.remove('Outcome_48Hours_Dispo')
features.remove('Arrival_Date')
features.remove('Rand')

In [10]:
# List of quantiles
nQ = 4     # Specify number of quantiles
dQ = 1.0/nQ  # Quantile increment
quantile_list = [x*dQ for x in range(nQ)]

In [11]:
# List of processed models (rules)

RuleList = []
RuleCount = 0

# Find best logical models
for nVar1 in range(len(features)):   # set first feature
    feat1 = features[nVar1]
    for nVar2 in range(nVar1+1, len(features)):  # set second covariate
        feat2 = features[nVar2]
        for val1, val1_top in [(covid_df[feat1].quantile(q), 
                                  covid_df[feat1].quantile(q+dQ)) for q in quantile_list]:
            
            for val2, val2_top in [(covid_df[feat2].quantile(q), 
                                  covid_df[feat2].quantile(q+dQ)) for q in quantile_list]:
            
                # Define new logical rule
                # rule = (df[var1]>val1) | (df[var2]>val2)
                decision_rule = (covid_df[feat1]>val1) & (covid_df[feat1]<val1_top) | (covid_df[feat2]>val2) & (covid_df[feat2]<val2_top) 

                # Increment Rule Count
                RuleCount = RuleCount + 1

                # Check if we have at least % of the population matching rule
                # Using a mask
                rule_df = covid_df[decision_rule]
                if rule_df.shape[0] < 0.01 * nRecords:
                    continue    # rare condition, skip

                # Compute fit metric of rule
                ModelFit = f1_score(covid_df['Outcome_48Hours_Dispo'], 
                                                    decision_rule)

                # Record new rule model
                rule_tuple = (feat1, feat2)
                RuleList.append((round(ModelFit, 4), rule_tuple))
            
            

            

In [12]:
print('Rules processed:', RuleCount)
RuleList.sort(key=lambda rule: -rule[0])

# Get lsit of quality values
fit_list = [f for f,_ in RuleList]

Rules processed: 448


When running logical rule models with the code explained in the class, what COVID risk pattern is identified by the most accurate rule?
- A)	High neutrophils or High respiration rate
- B)	High BMI and High neutrophils
- C)	High BMI or Low oxygen saturation
- D)	High age and High temperature


In [15]:
print("COVID risk pattern is identified by the most accurate rule:")
print(RuleList[0])

COVID risk pattern is identified by the most accurate rule:
(0.5182, ('AvgReading_Neuts_pct', 'Respiration_Rate'))
